In [260]:
import sys
import warnings

import matplotlib.pyplot as plt

import qiskit
from qiskit import IBMQ, QuantumCircuit, execute,  Aer
from qiskit.result import marginal_counts
from qiskit.providers.ibmq.job import job_monitor
from qiskit.tools.visualization import plot_histogram

In [261]:
IBMQ.load_account()

# Fill in your hub/group/provider
provider = IBMQ.get_provider(hub='ibm-q-afrl', group='air-force-lab', project='quantum-sim')
# ibmq_manhattan supports mid-circuit measurements
backend = provider.get_backend('ibm_lagos')

config = backend.configuration()
n_qubits = config.n_qubits

ibmqfactory.load_account:WARNING:2022-07-18 13:46:06,382: Credentials are already in use. The existing account in the session will be replaced.


# From Measurment

We need a function to measure the density mastrix.  We have $$ \rho = i I +z Z+ x X + y Y $$ so we can find each term by,
$$ 2i = \rho[0] + \rho[1] $$
$$ 2z = \rho[0] - \rho[1] $$
$$ 2x = R_y(\pi/2)\rho R_y(-\pi/2)[0] - R_y(\pi/2)\rho R_y(-\pi/2)[1] $$
$$ 2y = R_x(-\pi/2)\rho R_x(\pi/2)[0] - R_x(-\pi/2)\rho R_x(\pi/2)[1] $$

In [266]:
import copy
import numpy as np

sim = Aer.get_backend("qasm_simulator")
sim = provider.get_backend('ibm_lagos')

def measure_density(qc):
    qca = copy.deepcopy(qc)
    qca.measure([0], [0])
    r = execute(qca, backend = sim).result().get_counts()
    if '1' not in r: r['1'] = 0
    if '0' not in r: r['0'] = 0
    n = (r['0'] + r['1'])
    i = (r['0'] + r['1'])/2
    z = (r['0'] - r['1'])/2
    
    qcb = copy.deepcopy(qc)
    qcb.ry(np.pi/2,0)
    qcb.measure([0], [0])
    r = execute(qcb, backend = sim).result().get_counts()
    if '1' not in r: r['1'] = 0
    if '0' not in r: r['0'] = 0
    x = (r['0'] - r['1'])/2

    
    qcc = copy.deepcopy(qc)
    qcc.rx(np.pi/2,0)
    qcc.measure([0], [0])
    r = execute(qcc, backend = sim).result().get_counts()
    if '1' not in r: r['1'] = 0
    if '0' not in r: r['0'] = 0
    y = (r['0'] - r['1'])/2

    
    return [i/n,z/n,-x/n,y/n]
    
 


Using this we can reconstruct the density matrix from the results.  From the density matrix we can calculate the entropy.  Note the difference when applying a mid-circuit measurement.  

In [267]:
from Define_Paulis import I,X,Y,Z
import pandas as pd

import scipy.linalg as ln

def S(rho):
    e,y = ln.eig(rho)
    s = sum([np.round( (e[n]+10**-28)*np.log(e[n]+10**(-28)) , 8) for n in range(len(e))])
    return s

qc_simp = QuantumCircuit(1,1)
qc_simp.ry(np.pi/2,0)
qc_simp.measure([0], [0])

dx = measure_density(qc_simp)
rho = dx[0]*I(1) + dx[1]*Z(0,1) + dx[2]*X(0,1) + dx[3]*Y(0,1)

print(rho )
print()
print('entropy: ', S(rho))

[[0.51475+0.j     0.00825+0.0045j]
 [0.00825-0.0045j 0.48525+0.j    ]]

entropy:  (-0.6925353000000001+0j)


# From input

We can do the same thing by simpy changing the input of our circuit.  We will either start with $$ \rho = |0><0| $$ or with $$ \rho = |1><1| $$ and weight the results of both depending on the entropy.

In [264]:
qc_a = QuantumCircuit(1,1)
qc_b = QuantumCircuit(1,1)
qc_b.x(0)

da = measure_density(qc_a)
db = measure_density(qc_b)

rho_a = da[0]*I(1) + da[1]*Z(0,1) + da[2]*X(0,1) + da[3]*Y(0,1)
rho_b = db[0]*I(1) + db[1]*Z(0,1) + db[2]*X(0,1) + db[3]*Y(0,1)


print('entropy of A:',S(rho_a))
print('entropy of B:',S(rho_b))
print('entropy of A + B: ',S(0.9*rho_a + 0.1*rho_b))

entropy of A: (0.00897088-0.00363002j)
entropy of B: (0.0033230399999999998-0.00117402j)
entropy of A + B:  (-0.3221708+0j)


Then if we want to find the expectation value of an operator $O$ for a density matrix $$ \rho = \rho_a + \rho_b$$ we can simply add,
$$ \text{Tr}(O\rho) = \text{Tr}(O\rho_a) + \text{Tr}(O\rho_b)  $$

In [268]:
O = [[0.3,1.5],[1.5,-2.7]]

print( np.trace(np.dot(O,rho_a + rho_b)) )
print( np.trace(np.dot(O,rho_a)) + np.trace(np.dot(O,rho_b)) )

(-2.5054687500000004+0j)
(-2.5054687500000004+0j)
